In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Conv1D, MaxPooling1D, Flatten
import random

In [2]:
lemmatizer = WordNetLemmatizer()

data_file = open('../dataset/convo.json').read()
intents = json.loads(data_file)

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('../obj2/words.pkl','wb'))
pickle.dump(classes,open('../obj2/classes.pkl','wb'))

6402 documents
110 classes ['Abdonominal Pain', 'Abrasions', 'Broken Toe', 'Bruises', 'CPR', 'Choking', 'Cold', 'Cough', 'Cuts', 'Diarrhea', 'Drowning', 'Ethereal-useful', 'Fever', 'Fracture', 'Gastrointestinal problems', 'Headache', 'Nasal Congestion', 'Rash', 'Skin problems', 'Sore Throat', 'Splinter', 'Sprains', 'Strains', 'Wound', 'about', 'afternoon', 'animal bite', 'anxious', 'ask', 'attempt', 'casual', 'creation', 'death', 'default', 'depressed', 'done', 'emergency', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'jokes', 'learn-mental-health', 'learn-more', 'location', 'meditation', 'mental-health-fact', 'morning'

In [5]:
# Create our training data
training = []
# Create an empty array for our output
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Ensure the bag and output_row are of expected lengths
    if len(bag) != len(words) or len(output_row) != len(classes):
        print(f"Unexpected length found for document {doc}.")
        print(f"Bag length: {len(bag)}, Expected: {len(words)}")
        print(f"Output row length: {len(output_row)}, Expected: {len(classes)}")
    
    training.append([bag, output_row])

# Check if all elements in 'training' are of consistent shape
for i, item in enumerate(training):
    if len(item) != 2 or not isinstance(item[0], list) or not isinstance(item[1], list):
        print(f"Inconsistent item at index {i}: {item}")

# Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# Create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=10, verbose=1)
model.save('../models/chatbot_model.h5')

print("Model created")

d:\Main Folder\Campus\SEM 6\NLUG\projectai\myvenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Main Folder\Campus\SEM 6\NLUG\projectai\myvenv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3184 - loss: 3.1863
Epoch 2/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6515 - loss: 1.4109
Epoch 3/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7439 - loss: 1.0355
Epoch 4/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7981 - loss: 0.8422
Epoch 5/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8228 - loss: 0.7190
Epoch 6/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8348 - loss: 0.6487
Epoch 7/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8392 - loss: 0.6138
Epoch 8/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8503 - loss: 0.5909
Epoch 9/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8616 - loss: 0.5293
Epoch 10/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8732 - loss: 0.4858
Epoch 11/500
641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8791 - loss: 0.4770
Epoch 12/500
641/641 ━━━━━━━━━━━━━━━━━━━━

Model created
